Installing The Required Libraries

In [ ]:
pip install psycopg2-binary

In [ ]:
pip install IMDbPY

In [ ]:
pip install langchain

In [ ]:
pip install -qU langchain-openai

In [ ]:
pip install --quiet langchain langchain_core

In [ ]:
pip install --quiet langchain_text_splitters

In [ ]:
pip install --quiet faiss-cpu

In [ ]:
pip install --quiet langchain docarray tiktoken langchain_openai

In [35]:
pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# SQL Database Retreval

In [3]:
import psycopg2

def get_movie_data(movie_name):
    conn = None
    try:
        conn = psycopg2.connect(host="localhost", database="Project_DB", user="postgres", password="AS@5198974se")
        cursor = conn.cursor()
        query = 'SELECT "Rating", "Watched_Status" FROM "Movie_List" WHERE "Movie_Name" = %s'
        cursor.execute(query, (movie_name,))
        data = cursor.fetchall()
        if data:
            return [str(d) if d is not None else 'N/A' for d in data[0]]
        return ['N/A', 'N/A']
    finally:
        if conn:
            conn.close()

# IMDB Website Database Retrival

In [4]:
from imdb import IMDb

def get_movie_details(movie_name):
    ia = IMDb()
    search_results = ia.search_movie(movie_name)
    if search_results:
        movie = ia.get_movie(search_results[0].movieID)
        ia.update(movie, 'full credits')  # Get full credits for detailed cast info
        cast = ', '.join(str(person) for person in movie.get('cast', [])[:5])  # Safely convert cast members to string
        return {
            "title": str(movie.get('title', 'N/A')),
            "year": str(movie.get('year', 'N/A')),
            "directors": ', '.join(str(d) for d in movie.get('directors', [])),
            "genres": ', '.join(str(g) for g in movie.get('genres', [])),
            "cast": cast
        }

# Langchain Components

In [23]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

# Initialize Langchain components
openai_api_key = "sk-proj-P5QDwA3CwNApQYtepeYLT3BlbkFJ6T6yssZpdA8HLGp2LSwy"
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
model = ChatOpenAI(openai_api_key=openai_api_key)
output_parser = StrOutputParser()

# FAISS Index setup
dimension = 768  # This needs to match the output dimension of the embedding model
index = faiss.IndexFlatL2(dimension)
docstore = {}

# Add vectorizing code
def vectorize_data(text):
    """Vectorize the text data and store it in FAISS index but do not execute this automatically."""
    embeddings = generate_embeddings(text)
    if embeddings.ndim == 1:
        embeddings = np.expand_dims(embeddings, axis=0)
    index.add(embeddings)  # Add to FAISS index

def generate_embeddings(text):
    """Generate embeddings from text."""
    return np.array(embeddings.embed_documents(text))

# Define prompt template
template = """Please analyze the movie based on the following details:
Movie Title: {title}
Year: {year}
Genres: {genres}
Cast: {cast}
Directors: {directors}
Rating: {rating}
Watched Status: {watched_status}
Given this information, summarize the critical reception (based on rating out of 5 and watched status) and discuss the main themes of the film."""

prompt = ChatPromptTemplate.from_template(template)

# Setup Langchain pipeline correctly
setup_and_retrieval = RunnablePassthrough()
chain = setup_and_retrieval | prompt | model | output_parser

def answer_movie_question(movie_name):
    movie_details = get_movie_details(movie_name)
    movie_data = get_movie_data(movie_name)

    if not movie_details or not movie_data:
        return "Movie details or data not found."

    # Prepare the context with all necessary details
    context = {
        "title": movie_details['title'],
        "year": movie_details['year'],
        "genres": movie_details['genres'],
        "cast": movie_details['cast'],
        "directors": movie_details['directors'],
        "rating": movie_data[0] if movie_data else 'N/A',
        "watched_status": movie_data[1] if len(movie_data) > 1 else 'N/A'
    }

    try:
        # Invoke the chain with the context
        response = chain.invoke(context)  # The context now contains all required details for the template
        return response
    except Exception as e:
        return f"Error processing the query: {str(e)}"

# Example usage
print(answer_movie_question("Lou"))

Based on the 4.0 rating and the fact that the movie has not been watched, it seems that "Lou" has received positive critical reception. The film likely explores a mix of genres, including action, adventure, crime, drama, mystery, and thriller, which suggests a complex and engaging plot. 

The main themes of "Lou" could include elements of mystery and crime, as well as themes of adventure and action. The diverse genres suggest a multi-layered storyline that may involve a thrilling mystery, high-stakes crime, and intense drama. The talented cast, including Allison Janney, Jurnee Smollett, and Logan Marshall-Green, further indicates that the film likely offers strong performances and engaging characters. Overall, "Lou" appears to be a well-received movie that promises an exciting and suspenseful viewing experience.
